In [107]:
import networkx as nx
import operator

In [225]:
#transofrms a directed cyclic graph to directed acyclic graph by compressing the components of the graph into one node
def transformToDAG(G):
    g2 = nx.DiGraph()
    components = list(nx.strongly_connected_components(G))
    for component in components:
        new_node = ','.join(str(e) for e in component)
        if not(new_node in g2.nodes):
            g2.add_node(new_node, size=len(component))
    
        suc_nodes = set()
        for e in component:
            for nei in G.neighbors(e):
                if not (nei in component):
                    suc_nodes.add(nei)
        for node in suc_nodes:
            for c in components:
                if node in c:
                    to_n = ','.join(str(e) for e in c)
                    break
            if not(to_n in g2.nodes):
                g2.add_node(to_n, size=len(component))
            if not((new_node, to_n) in g2.edges):
                g2.add_edge(new_node, to_n, weight=g2.node[to_n]['size'])
    return g2
    

In [199]:
#finds all the bridges based on importance --> based on their weight and returns a soreted list of couples of edge and weigth
def getBridges(G):
    bridges = {}
    for brg in nx.bridges(G.to_undirected()):
        if (brg[1], brg[0]) in G.edges:
            bridges[(brg[1], brg[0])] = G.get_edge_data(brg[1], brg[0], 'weight')['weight']
        else:
            bridges[brg] = G.get_edge_data(brg[0], brg[1], 'weight')['weight']
    sorted_brgs = sorted(bridges.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_brgs

In [286]:
def getLongestPath(G):
    return nx.dag_longest_path(G)

In [272]:
#I changed the weights to minus ones but it did not work to give me the longest path
#myg2 = nx.DiGraph()
#for e in myg.edges:
#    myg2.add_edge(e[0], e[1], weight=(-1 * myg[e[0]][e[1]]['weight']))
#nx.draw(myg2, with_labels=True)

In [335]:
#find shortest (longest) paths between the root and all leaves --> as parallel paths
def calculatePathWeight(G, path):
    w = 0
    i = 0
    while i < (len(path) - 1):
        w = w + G[path[i]][path[i+1]]['weight']
        i = i + 1
    return w

def getParallelPaths(G):
    leaves = []
    roots = []
    for n in G.nodes:
        if G.out_degree(n) == 0:
            leaves.append(n)
        if G.in_degree(n) == 0:
             roots.append(n)
    paths = []
    for l in leaves:
        for r in roots:
            if not l in nx.descendants(G, r):
                continue
            p = nx.shortest_path(G, r, l, weight='weight')
            paths.append((p, calculatePathWeight(G, p)))
    paths.sort(key=operator.itemgetter(1), reverse=True)
    return paths

In [203]:
#finds all parallel sequences of nodes, returns as sets of parallel paths
#def get_parallel_paths(G):
#    return [list(nx.all_simple_paths(G, i, j)) for i in G.nodes for j in G.nodes if i != j and nx.has_path(G, i, j)]

#def dfsTree(n, G, path, paths):
#    path.append(n)
#    if len(list(G.neighbors(n))) == 0:
#        paths.append(path.copy())
#    for nei in G.neighbors(n):
#        dfsTree(nei, G, path, paths)
#    path.pop()
#    return paths

#mytree = nx.maxi_spanning_tree(myg.to_undirected())
#nx.draw(mytree, with_labels=True)

#for x in nx.dag_to_branching(myg):
#print(x)

#from collections import defaultdict
#from operator import itemgetter

#G = myg
#B = nx.dag_to_branching(G)

#sources = {}
#visited = []
#i = 1
#for v, source in B.nodes(data='source'):
#    if source not in visited:
#        sources[v] = source
#        visited.append(source)
#    else:
#        sources[v] = source + str(i)
#        i = i + 1
#B = nx.relabel_nodes(B, sources)
#root = None
#for v in B:
#    if B.in_degree(v) == 0:
#        root = v
#        break

        
#y = dfsTree('1', myg, [], [])
#for x in y:
#    print(x)

#nx.draw(B, with_labels=True)